# Traitement des données pour l'intelligence artificielle et la santé
**Author:** [LudovicSaintBauzel](https://www.isir.upmc.fr/personnel/saintbauzel/)<br>
**Date created:** 2023/06/01<br>
**Last modified:** 2023/06/06<br>
**Description:** Atelier sur l'apprentissage et le traitement des données pour le "summer school" Intelligence Artificielle et santé.

<a href="https://colab.research.google.com/github/LudovicSaintBauzel/AI4Health/blob/main/SummerSchool2023_IA_Sante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Initialisation des librairies pour le travail.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
import os
import glob

# Chargement des données

Récupérons les données d'une experience d'assistance au levé robotisé. 

In [ ]:
!curl -O https://nuage.isir.upmc.fr/index.php/s/ZiPJ3RFGFaF3HBs/download/records.zip

# Analyse et travail sur les données

In [ ]:
!unzip -o records.zip
!ls 

## Observons l'organisation des données

Maintenant que les données sont téléchargées, il est possible de connaître l'organisation des données en ouvrant le fichier (/content/records/DESCRIPTIONS_DONNEES.txt). On note que le fichier follow_traj.txt contient toutes les informations utiles pour la classification que nous souhaitons effectuer. 


In [ ]:
os.chdir('/Users/ludo/Documents/src/AI4Health')

# Liste des noms de fichiers CSV
fichiers_csv = [[file for file in glob.glob("*/*/*/follow_traj.txt")]]

Une vision naïve du chargement des données 

In [ ]:
donnees = []

# Charger chaque fichier CSV et ajouter les données à la liste
for fichier_csv in fichiers_csv[0]:
    print(fichier_csv)
    data = pd.read_csv(fichier_csv, sep='\s+', header=None)
    donnees.append(data)

# Concaténer toutes les données en un seul DataFrame
donnees_combinees = pd.concat(donnees)


En fait il existe une quantité de raison pour que les fichiers soient corrompus. Dans notre cas certains fichiers sont vides. ```stat st_size ```


In [ ]:
# Liste pour stocker les données de tous les fichiers
donnees = []

filtering_data = ['']
#'records/test1/traj/follow_traj.txt',
#                  'records/BO2504/test/follow_traj.txt',
#                  'records/fuzzy/tt/follow_traj.txt',
#                  'records/test1/fx/follow_traj.txt',
#                  # Above Problematic data : Below test data not relevant
#                  'records/test1/av/follow_traj.txt',
#                  'records/test2/traj/follow_traj.txt',
#                  'records/test2/av/follow_traj.txt',
#                  'records/traj/test/follow_traj.txt',
#                  'records/move/follow_traj.txt'
#                  ]

# Charger chaque fichier CSV et ajouter les données à la liste
for fichier_csv in fichiers_csv[0]:
    
    ??? 
    if (???) and (fichier_csv not in filtering_data):
        print(fichier_csv)
        data = pd.read_csv(fichier_csv, sep='\s+', header=None)
        donnees.append(data)
    else:
        print("File filtered not loaded : "+str(fichier_csv)+"\n")

# Concaténer toutes les données en un seul DataFrame
donnees_combinees = pd.concat(donnees)


Desfois il faut aussi supprimer des lignes aberrantes. Retrait des lignes qui contiennent "not a number". 

``` dropna ```

In [ ]:

donnees_combinees = ???


# Apprentissage des données

## Construction des données de validation et de test

Diviser les données en features (X) et labels (y). Par exemple prenons les forces des poignées en entrées (features X)  et la sortie consRob en sortie. Pour cela on doit reprendre les bonnes colonnes grâce au fichier (/content/records/DESCRIPTIONS_DONNEES.txt).


In [ ]:
X = donnees_combinees.iloc[:, ???:???].values
y = donnees_combinees.iloc[:, ???].values

Diviser les données en ensembles d'entraînement et de test. ```train_test_split ```

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = ???

On peut ensuite construire un scaler sur les données de train et l'appliquer aux données de test. 

```StandardScaler fit_transform```

In [ ]:

from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
# Normaliser les données
???
X_train = ???
X_test = ???

# Normaliser les données de sortie
min_y_train = min(y_train)
range_y_train = max(y_train) - min(y_train)
y_train = (y_train)/range_y_train
y_test = (y_test)/range_y_train


# Apprentissage 
## Création du modèle de réseau de neurones

Comme expliqué il est important de comprendre l'enjeux des paramètres de notre modèle. Le point à garder à l'esprit est le nombre de données qui sont embarqué dans le réseau. Et de se garantir du pouvoir généralisant de la méthode. Etant donnée le fonctionnement de l'algorithme de rétropropagation, on peut difficilement envisager d'avoir moins de 10 données pour un lien dans le réseau. 


In [ ]:
import tensorflow as tf
# Créer le modèle du réseau de neurones
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(
    20, activation='sigmoid', input_shape=(X_train.shape[1],)))
# model.add(tf.keras.layers.Dense(4, activation='sigmoid'))

#model.add(tf.keras.layers.Dense(12, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

## Apprentissage du modèle

In [ ]:
# Entraîner le modèle
#
epochs = 20
callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]

history = model.fit(X_train, y_train, epochs=epochs,
                    batch_size=8, callbacks=callbacks, validation_data=(X_test, y_test))


## Affichage de l'évolution de l'apprentissage et de la sortie du système

In [ ]:
import matplotlib.pyplot as plt 
# plot
accuracy = history.history['accuracy']
loss = history.history['loss']
val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

# Tracer la précision
plt.plot(accuracy, label='Entraînement')
plt.plot(val_accuracy, label='Validation')
plt.title('Précision du modèle')
plt.xlabel('Époque')
plt.ylabel('Précision')
plt.legend()
plt.show()

# Tracer la perte
plt.plot(loss, label='Entraînement')
plt.plot(val_loss, label='Validation')
plt.title('Perte du modèle')
plt.xlabel('Époque')
plt.ylabel('Perte')
plt.legend()
plt.show()
# Tracer la performance du modèle

# Évaluer le modèle
yres = model.predict(X_test)
plt.plot(yres[0:100], label='Prédiction')
plt.plot(y_test[0:100], label='Réel')
plt.title('Performance du modèle')
plt.xlabel('Échantillon')
plt.ylabel('Valeur')
plt.legend()
plt.show()

# regression

In [ ]:
from sklearn.linear_model import ElasticNet
# Créer un regressor ElasticNet
regressor = ElasticNet(alpha=0.1, l1_ratio=0.5)

# Entraîner le modèle
#
epochs = 20
callbacks = [
    #tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.keras"),
]
    
# Entraîner le regressor sur les données d'entraînement
res = regressor.fit(X_train, y_train)
res


In [ ]:
# Faire des prédictions sur les données de test
predictions = regressor.predict(X_test)

plt.plot(predictions[0:100], label='Prédiction')
plt.plot(y_test[0:100], label='Réel')
plt.title('Performance du modèle')
plt.xlabel('Échantillon')  
plt.ylabel('Valeur')
plt.legend()
plt.show()